In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
l = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        l.append(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
l

# 1. Import data

In [ ]:
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
shops_data = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
sales_train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
sample_submission = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
item_category = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")

# **2. Exploratory data analysis** 

If you translate parts of the shops names it looks like the characters before the " are cities in russia with a set of letters at the end, which can be some type of each shop. In this case, I found 32 cities with shops and 10 types of shop.


In [ ]:
cities = []
types = []

for i in shops_data.shop_name.str.split('"'):
    j = i[0].split(' ')
    cities.append(j[0])
    types.append(j[1])
cities = np.array(cities)
types = np.array(types)

non = np.array(['', '(Плехановская,', 'Магазин', 'Орджоникидзе,', 'Посад'])
types = np.where(np.isin(types,non), 'None', types)
shops_data['city'] = cities
shops_data['type_shop'] = types 

In the item_category we found that the first word of each category is repeated several times. In this case, I found 15 unique values, which can be used to reduce the number of categories for each item

In [ ]:
first = []
for i in item_category.item_category_name.str.split(" "):
    first.append(i[0])
item_category['new_category'] = first
items = items.merge(item_category, on='item_category_id', how='left')

**Graphs of distribution between test and train data set (test, sales_train)**

In [ ]:
plt.figure(figsize=(13,5))

plt.subplot(121)
plt.scatter(sales_train.shop_id, sales_train.item_id)
plt.xlabel('shop_id')
plt.ylabel('item_id')
plt.title('train_data')


plt.subplot(122)
plt.scatter(test.shop_id, test.item_id)
plt.xlabel('shop_id')
plt.ylabel('item_id')
plt.title('test_data')
plt.show()

In [ ]:
plt.figure(figsize=(13,10))

plt.subplot(221)
plt.hist(sales_train.shop_id, bins=25)
plt.xlabel('shop_id')
plt.ylabel('counts')
plt.title('train_data')

plt.subplot(222)
plt.hist(test.shop_id, bins=25)
plt.ylabel('counts')
plt.xlabel('shop_id')
plt.title('test_data')

plt.subplot(223)
plt.hist(sales_train.item_id, bins=25)
plt.xlabel('item_id')
plt.ylabel('counts')
plt.title('train_data')

plt.subplot(224)
plt.hist(test.item_id, bins=25)
plt.ylabel('counts')
plt.xlabel('item_id')
plt.title('test_data')

plt.show()

Lets see how many unique values both data set have

In [ ]:
test.nunique()

In [ ]:
sales_train.nunique()

The aoumnt of values for the test set is equal to all the possibl ecombinations between all the shops and all the items that this data set have. For this reason, I should take into account all the possible combinations for shop_id and item_id per month in the train data set

In [ ]:
sales_train['date'] = pd.to_datetime(sales_train.date, format='%d.%m.%Y')
sales_train['year'] = sales_train.date.dt.year
sales_train['month'] = sales_train.date.dt.month

In [ ]:
sales_train = sales_train[sales_train.shop_id.isin(test.shop_id.unique())]
sales_train = sales_train[sales_train.item_id.isin(test.item_id.unique())]

I used the new set of categories in order to make graphs easier to read

In [ ]:
sales_train = sales_train.merge(items[['item_id', 'item_category_id', 'new_category']], on='item_id', how='left')
sales_train = sales_train.merge(shops_data[['city', 'type_shop', 'shop_id']], on='shop_id', how='left')

sales_train.item_cnt_day = sales_train.item_cnt_day.fillna(0).clip(0,20)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

shops = sales_train.shop_id.unique()
shops = np.sort(shops)
pos = 1
plt.figure(figsize=(20,85))

for j in shops:
    k = sales_train[sales_train.shop_id == j].groupby(['new_category', 'date_block_num'], as_index=False).sum()

    for i in k.new_category.unique():
        x = k.date_block_num[k.new_category == i]
        y = k.item_cnt_day[k.new_category == i]    
        plt.subplot(int(len(shops)/3), 3, pos)     
        if j == 36:
            plt.scatter(x,y, label=i)
        else:
            plt.plot(x,y, label=i)
    
    m = sales_train[sales_train.shop_id == j].groupby('date_block_num', as_index=False).sum()
    y_t = m.item_cnt_day
    x_t = m.date_block_num.unique()
    if j == 36:
        plt.scatter(x_t, y_t, label='total', color='black' )
    else:
        plt.plot(x_t, y_t, label='total', color='black' )
    plt.xlabel('time')
    plt.ylabel('quantity')
    plt.xticks(np.arange(0, 33, 3))
    plt.title("shop_id = "+ str(j))
    plt.legend()
    pos += 1
    
plt.show()

Here, there is a graph of sold items per month in each store. In each case, the months closer to the end of the year show an increase in sales. Also we can observe the openning of some stores and a peculiar case, in the trainnig data we only have data for the 33 month of the shop with Id=36. In addition, as it is expected, that the sales show a season dependance, maybe there are products that are more popular in specific parts of the year

In [ ]:
plt.figure(figsize=(20,110))
pos = 1
for i in np.sort(sales_train.item_category_id.unique()):
    plt.subplot(20,3,pos)
    x = sales_train.date_block_num[sales_train.item_category_id == i].unique()
    x = np.sort(x)
    y = sales_train[sales_train.item_category_id == i].groupby('date_block_num').sum().item_cnt_day
    y /= np.linalg.norm(y)
    plt.plot(x,y, label='cnt')
    
    y = sales_train[sales_train.item_category_id == i].groupby('date_block_num').mean().item_price
    y_std = sales_train[sales_train.item_category_id == i].groupby('date_block_num').std().item_price.values
    y_std /= np.linalg.norm(y)
    y /= np.linalg.norm(y)
    plt.plot(x,y, label='price')
    plt.errorbar(x,y, yerr=y_std)
    plt.xlabel('time')
    plt.ylabel(' normalized quantity / average price')
    plt.xticks(np.arange(0, 33, 3))
    plt.title('item_category_id = ' + str(i))
    plt.legend()
    pos += 1
plt.show()

Here we evaluate the behaviour of the sales and the price for the 60 categories_id in the training data set. In this case, we observe the releasing of some new categories. In addition, sometimes the change of average prices occurs at the same month of change in sales. In the graph bellow, the error lines are equivalent to the standard deviation of the price for each category per month

Now lets make a graph of the change in price and the sales for each category. In this case, the change and the price of each item could be usefull for the models. Because we observe a change in both feauteres at the same time of a change in the sales

In [ ]:
plt.figure(figsize=(20,110))
pos = 1
for i in np.sort(sales_train.item_category_id.unique()):
    plt.subplot(20,3,pos)
    x = sales_train.date_block_num[sales_train.item_category_id == i].unique()
    x = np.sort(x)
    y = sales_train[sales_train.item_category_id == i].groupby('date_block_num').sum().item_cnt_day
    y /= np.linalg.norm(y)
    plt.plot(x,y, label='cnt')
    
    y = sales_train[sales_train.item_category_id == i].groupby('date_block_num').mean().item_price
    y = np.diff(y)
    y /= np.linalg.norm(y)
    plt.plot(x[:-1],y, label='price')
    plt.xlabel('time')
    plt.ylabel(' normalized quantity /  change on average price')
    plt.xticks(np.arange(0, 33, 3))
    plt.title('item_category_id = ' + str(i))
    plt.legend()
    pos += 1
plt.show()

Lets extract the year, month, day and day of the week for all the dataset

In [ ]:
sales_train['day'] = sales_train.date.dt.day
sales_train['day_week'] = sales_train.date.dt.dayofweek

Now lets graph the amount of sales for each month per year

In [ ]:
#plt.figure(figsize=(20,110))
p = [1, 0, -1]
pos = 0
for i in np.sort(sales_train.year.unique()):
    x = sales_train.month[sales_train.year == i].unique()
    x = np.sort(x)
    y = sales_train[sales_train.year == i].groupby('month').sum().item_cnt_day
    #plt.barh(x,y, label='year '+ str(i), alpha=0.5)
    
    #x = np.arange(len(x))  # the label locations
    width = 0.4  # the width of the bars

        
    plt.bar(x + p[pos] * width/2, y, width/2, label='year '+ str(i))
    pos += 1
    
    plt.xlabel('month')
    plt.ylabel('quantity')
    plt.legend()
plt.show()

Lets plot for each day of the month

In [ ]:
plt.figure(figsize=(35,20))
pos = 1
for j in np.sort(sales_train.year.unique()):
    h = sales_train[sales_train.year == j]
    plt.subplot(2,2,pos)
    for i in np.sort(sales_train.month.unique()):
        x = h.day[h.month == i].unique()
        x = np.sort(x)
        y = h[h.month == i].groupby('day').sum().item_cnt_day
        plt.bar(x,y, label='month '+ str(i), alpha=0.5)
        plt.xlabel('day')
        plt.ylabel('quantity')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title('year ' + str(j))
    pos += 1
plt.show()

Now lets evaluate the sales per day of week for each month

In [ ]:
plt.figure(figsize=(26,22))
p = [1, 0, -1]
pos_ = 1
labels = [ 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday' ]
for i in np.sort(sales_train.month.unique())[:10]:
    h = sales_train[sales_train.month == i]
    plt.subplot(4, 3, pos_)
    pos = 0
    for j in np.sort(sales_train.year.unique()):
        x = h.day_week[h.year == j].unique()
        x = np.sort(x)
        y = h[h.year == j].groupby('day_week').sum().item_cnt_day
        
        x = np.arange(len(labels))  # the label locations
        width = 0.4  # the width of the bars

        
        plt.bar(x + p[pos] * width/2, y, width/2, label='year '+ str(j))
        pos += 1
        
        #plt.barh(x,y, label='year '+ str(j), alpha=0.5)
    plt.xlabel('day of the week')
    plt.ylabel('quantity')
    plt.legend()#bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.xticks(x, labels)
    plt.title('month ' + str(i))
    pos_ += 1
plt.show()

With all the graphs it looks for some months in the weekends are more sales in all the years of the training set. In addition, for the two first years the last days of the month showed an increase in the sales. Finally, at the end of each year the sales increase.

# 3. Featuring processing, train/validation split & model fitting

In this case, I created a lot of lag features with the average of the price, the sales, the average of the sales per combination date/item_id and per combination date/shop_id. In addition, I built a set of features that  independt of time like the average sale per combination city/new_category, per combination city/category_id, per combination type_shop/new_category, per combination type_shop/category_id. Later, I used the last month of the given training set as the validation set and trained a XGBRegressor with early stopping. All this procedure can be seen in [Featuring processing and model fitting](https://www.kaggle.com/dagarzon1/features-and-model)